## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Data links

DBJ meeting (Datawrapper table with data partway down the page): https://www.bizjournals.com/dallas/news/2023/05/25/vacant-land-solve-housing-rentcafe-study.html

Tiger Shapefile links: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2022&layergroup=ZIP+Code+Tabulation+Areas

## Data read-in

In [3]:
zip_codes = gpd.read_file('tl_2022_us_zcta520.zip')

In [4]:
df = pd.read_csv('Dallas vacant land by zip code - All Zip Codes.csv')

In [5]:
df['Zip Code'] = df['Zip Code'].astype(str)
zip_codes['ZCTA5CE20'] = zip_codes['ZCTA5CE20'].astype(str)

In [6]:
merged_df = pd.merge(left=df, right=zip_codes, left_on='Zip Code',right_on='ZCTA5CE20',how='inner')

In [7]:
merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

In [8]:
merged_df.columns

Index(['Zip Code', 'No. of Vacant Parcels', 'Vacant Land Area (Sq.Ft.)',
       'Estimated Units  ', 'ZCTA5CE20', 'GEOID20', 'CLASSFP20', 'MTFCC20',
       'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20',
       'geometry'],
      dtype='object')

In [11]:
def popup_html(row):
    i = row
    zip_code = merged_df['Zip Code'].iloc[i]
    VP = merged_df['No. of Vacant Parcels'].iloc[i]
    VLA = df['Vacant Land Area (Sq.Ft.)'].iloc[i]
    EU = merged_df['Estimated Units  '].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Zip Code: </strong>{}'''.format(zip_code) + '''<br>
    <strong>No. of Vacant Parcels: </strong>{}'''.format(VP) + '''<br>
    <strong>Vacant Land Area (Sq.Ft.): </strong>{}'''.format(VLA) + '''<br>
    <strong>Estimated Units: </strong>{}'''.format(EU) + '''<br>
    </html>
    '''
    return html

In [12]:
# Create a folium map object
m = folium.Map(location=[32.7767,-96.7970], zoom_start=10)

# Title HTML for the map
title_html = '''
<h3 align="center" style="font-size:16px"><b>{}</b></h3>
'''.format('Dallas Vacant Land')

# Add the title HTML to the map
m.get_root().html.add_child(folium.Element(title_html))

# Iterate over each row in the DataFrame
for i in range(len(merged_df)):
    # Generate the popup content
    html = popup_html(i)
    
    # Create a folium Popup object
    popup = folium.Popup(html, max_width=250)
    
    # Create a folium GeoJson object with the polygon and popup
    folium.GeoJson(merged_df['geometry'].iloc[i], popup=popup).add_to(m)

# Display the map
m

In [10]:
m.save('index.html')

## Map URL Snagger

In [11]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Dallas_vacant_land
